## Introduction to B ##

### Basic Datavalues ###

B provides the booleans, strings and integers as built-in datatypes.

In [1]:
BOOL

{FALSE,TRUE}

In [2]:
"this is a string"

"this is a string"

In [3]:
1024

1024

Users can define their own datatype in a B machine.
One distinguishes between explicitly specified enumerated sets and deferred sets.

In [5]:
::load
MACHINE MyBasicSets
SETS Trains = {thomas, gordon}; Points
END

[2018-05-25 13:12:41,869, T+1176691] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-05-25 13:12:43,407, T+1178229] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 54757
[2018-05-25 13:12:43,408, T+1178230] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 94534
[2018-05-25 13:12:43,412, T+1178234] "ProB Output Logger for instance 6e2e7ba7" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-05-25 13:12:43,441, T+1178263] "ProB Output Logger for instance 6e2e7ba7" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: MyBasicSets : []


For animation and constraint solving purposes, ProB will instantiate deferred sets to some finite set (the size of which can be controlled).

In [6]:
Points

{Points1,Points2}

### Pairs ###
B also has pairs of values, which can be written in two ways:

In [7]:
(thomas,10)

(thomas↦10)

In [8]:
thomas |-> 10

(thomas↦10)

Tuples simply correspond to nested pairs:

In [12]:
(thomas |-> gordon |-> 20)

((thomas↦gordon)↦20)

### Sets ###
Sets in B can be specified in multiple ways.
For example, using explicit enumeration:

In [13]:
{1,3,2,3}

{1,2,3}

or via a predicate by using a set comprehension:

In [14]:
{x|x>0 & x<4}

{1,2,3}

For integers there are a variety of other sets, such as intervals:

In [15]:
1..3

{1,2,3}

or the set of implementable integers INT = MININT..MAXINT or the set of implementable natural numbers NAT = 0..MAXINT.

Sets can be higher-order and contain other sets:

In [17]:
{ 1..3,  {1,2,3,2}, 0..1, {x|x>0 & x<4} }

{{0,1},{1,2,3}}

Relations are modelled as sets of pairs:

In [18]:
{ thomas|->gordon, gordon|->gordon, thomas|->thomas}

{(thomas↦thomas),(thomas↦gordon),(gordon↦gordon)}

Functions are relations which map every domain element to at most one value:

In [22]:
{ thomas|->1, gordon|->2}

{(thomas↦1),(gordon↦2)}

## Expressions vs Predicates vs Substitutions ##


### Expressions ###
Expressions in B have a value. With ProB and with ProB's Jupyter backend, you can evaluate expresssions such as:

In [19]:
2**1000

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376

B provides many operators which return values, such as the usual arithmetic operators but also many operators for sets, relations and functions.

In [21]:
(1..3 \/ 5..10) \ (2..6)

{1,7,8,9,10}

In [23]:
ran({(thomas↦1),(gordon↦2)})

{1,2}

In [24]:
{(thomas↦1),(gordon↦2)} (thomas)

1

In [25]:
{(thomas↦1),(gordon↦2)}~[2..3]

{gordon}

## Predicates
ProB can also be used to evaluate predicates (B distinguishes between expressions which have a value and predicates which are either true or false).

In [26]:
2>3

FALSE

In [27]:
3>2

TRUE

Within predicates you can use **open** variables, which are implicitly existentially quantified.
ProB will display the solution for the open variables, if possible.

In [28]:
x*x=100

TRUE

Solution:
	x = −10

We can find all solutions to a predicate by using the set comprehension notation.
Note that by this we turn a predicate into an expression.

In [29]:
{x|x*x=100}

{−10,10}

### Substitutions ###
B also has a rich syntax for substitutions, aka statements.
For example ```x := x+1``` increments the value of x by 1.
We will not talk about substitutions in the rest of this presentation.

## Constraint Solving ##

Constraint solving is determine whether a predicate with open/existentially quantified variables is satisfiable and providing values for the open variables in case it is.
We have already solved the predicate ```x*x=100``` above, yielding the solution ```x=-10```.
The following is an unsatisfiable predicate:

In [31]:
x*x=1000

FALSE

# Constraint solving has many applications in formal methods in general and B in particular.
It is required to animate implicit specifications.
Take for example an event
```
train_catches_up = any t1,t2,x where t1:dom(train_position) & t2:dom(train_position) &
                                     train_position(t1) < train_position(t2) &
                                     x:1..(train_position(t2)-train_position(t1)-1) then
                         train_position(t1) := train_position(t1)+x end
```
To determine whether the event is enabled, and to obtain values for the parameters of the event in a given state of the model, we have to solve the following constraint:

In [39]:
train_position = {thomas|->100, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦100),(gordon↦2020)}
	t1 = thomas
	t2 = gordon

In [40]:
train_position = {thomas|->2019, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

FALSE

Suppose that we have the invariant, ```train_position:TRAINS-->1..10000``` we can check whether the event is feasible in at least one valid state by solving:

In [38]:
train_position:Trains-->1..10000 &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦1),(gordon↦3)}
	t1 = thomas
	t2 = gordon

Many other applications exist: generating testcases, finding counter examples using bounded model checking or other algorithms like IC3.
Other applications are analysing proof obligations.
Take the proof obligation for an event theorem t1 $/=$ t2:
```
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) |- t1 /= t2
```
We can find counter examples to it by negating the proof goal:

In [42]:
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) & not( t1 /= t2 )

FALSE

Obviously, we can also use constraint solving to solve puzzles or real-life problems.
#### Send More Money Puzzle ####
We now try and solve the SEND+MORE=MONEY arithmetic puzzle in B, involving 8 distinct digits:

In [43]:
{S,E,N,D, M,O,R, Y} <: 0..9 & S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
  M*10000 + O*1000 + N*100 + E*10 + Y

TRUE

Solution:
	R = 8
	S = 9
	D = 7
	E = 5
	Y = 2
	M = 1
	N = 6
	O = 0

We can find all solutions (to the unmodified puzzle) using a set comprehension and make sure that there is just a single soltuion:

In [44]:
  {S,E,N,D, M,O,R, Y |
   {S,E,N,D, M,O,R, Y} <: 0..9 &  S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
   M*10000 + O*1000 + N*100 + E*10 + Y }

{(((((((9↦5)↦6)↦7)↦1)↦0)↦8)↦2)}

#### KISS PASSION Puzzle####
A slightly more complicated puzzle (involving multiplication) is the KISS * KISS = PASSION problem.

In [46]:
   {K,P} <: 1..9 &
    {I,S,A,O,N} <: 0..9 &
    (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S) 
     =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N &
    card({K, I, S, P, A, O, N}) = 7

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

Finally, a simple puzzle involving sets is to find a subset of numbers from 1..5 whose sum is 14:

In [48]:
x <: 1..5 & SIGMA(y).(y:x|y)=14

TRUE

Solution:
	x = {2,3,4,5}

## How to solve (set) constraints in B ##

### Booleans ###

If we have only booleans, constraint solving is equivalent to SAT solving.
Internally, ProB has an interpreter which does not translate to CNF (conjunctive normal form), but is otherwise similar to DPLL: deterministic propagations are carried out first (unit propagation) and there are heuristics to choose the next boolean variable to enumerate.

#### Knights and Knave Puzzle####
Here is a puzzle from Smullyan involving an island with only knights and knaves.
We know that:
 - Knights: always tell the truth
 - Knaves: always lie

We are given the following information about three persons A,B,C on the island:
 1. A says: “B is a knave or C is a knave”
 2. B says “A is a knight”

What are A, B and C?
Note: we model A,B,C as boolean variables which are equal to TRUE if they are a knight and FALSE if they are a knave.

In [49]:
 (A=TRUE <=> (B=FALSE or C=FALSE)) & // Sentence 1
 (B=TRUE <=> A=TRUE) // Sentence 2

TRUE

Solution:
	A = TRUE
	B = TRUE
	C = FALSE

### Integers ###

Let us take the integer constraint ```x*x=100``` which we saw earlier.
This constraint is actually more complicated than might appear at first sight: the constraint is not linear and the domain of x is not bounded. Indeed, B supports mathematical integers without any bit size restriction.
So, let us first look at some simpler constraints, where the domains of the variables are all bounded.
Let us consider a small arithmetic puzzle 
```
  X Y
+ X Y
-----
  Y 0
```

In [7]:
X:1..9 & Y:0..9 & X*10 + Y + X*10 + Y = Y*10

TRUE

Solution:
	X = 2
	Y = 5

Given that we know the domain of X and Y one can represent the integers by binary numbers and convert the constraint to a SAT problem.

Let us look at an even simpler constraint X:0..3 & Y:0..3 & X+Y=2. As you can see there are three solutions for this constraint:


In [14]:
{X,Y|X:0..3 & Y:0..3 & X+Y=2}

{(0↦2),(1↦1),(2↦0)}

We will now study how such constraints can be solved.

#### Solving constraints by translating to SAT ####
Given that we know the domain of X and Y in the constraint ``` X:0..3 & Y:0..3 & X+Y=2``` one can represent the integers by binary numbers and convert the constraint to a SAT problem.
The number 2 is 10 in binary and we can represent X and Y each by two bits X0,X1 and Y0,Y1.
We can translate the addition to a propositional logic formula:

Bit1 | Bit0
-----|-----
   X1  |  X0
 + Y1  |  Y0
     |
 Z1  |  Z0
 
Let us find one solution to this constraint, by encoding addition using an additional carry bit:

In [12]:
 ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
 ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
 (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) &
 (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
 Z0=FALSE & Z1=TRUE

TRUE

Solution:
	Z0 = FALSE
	Y0 = TRUE
	Z1 = TRUE
	X0 = TRUE
	Y1 = TRUE
	X1 = TRUE
	CARRY0 = TRUE

In [16]:
{X0,X1,Y0,Y1,Z0,Z1,CARRY0 | ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
   ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
   (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) & 
   (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
    Z0=FALSE & Z1=TRUE}

{((((((FALSE↦FALSE)↦FALSE)↦TRUE)↦FALSE)↦TRUE)↦FALSE),((((((FALSE↦TRUE)↦FALSE)↦FALSE)↦FALSE)↦TRUE)↦FALSE),((((((TRUE↦FALSE)↦TRUE)↦FALSE)↦FALSE)↦TRUE)↦TRUE),((((((TRUE↦TRUE)↦TRUE)↦TRUE)↦FALSE)↦TRUE)↦TRUE)}

As you can see, we have found four solutions and not three! One solution is 3+3=2.
This is a typical issue when translating arithmetic to binary numbers: we have to prevent overflows.

In [20]:
{X0,X1,Y0,Y1,Z0,Z1,CARRY0 | ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
   ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
   (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) & 
   (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
   (CARRY0=TRUE => (X1=FALSE & Y1=FALSE)) & // no overflow
   (CARRY0=FALSE => (X1=FALSE or Y1=FALSE)) & // no overflow
    Z0=FALSE & Z1=TRUE}

{((((((FALSE↦FALSE)↦FALSE)↦TRUE)↦FALSE)↦TRUE)↦FALSE),((((((FALSE↦TRUE)↦FALSE)↦FALSE)↦FALSE)↦TRUE)↦FALSE),((((((TRUE↦FALSE)↦TRUE)↦FALSE)↦FALSE)↦TRUE)↦TRUE)}

In ProB, we can use **Kodkod** backend to achieve such a translation to SAT.
- Kodkod is the API to the **Alloy** constraint analyzer and takes relational logic predicates and translates them to SAT.
- The SAT problem can be solved by any SAT solver (Sat4J, minisat, glucose,...).
- ProB translates parts of B to the Kodkod API and translates the results back to B values.
- Prior to the translation, ProB performs an interval analysis to determine possible ranges for the integer decision variables.

The details were presented at FM'2012.

In [21]:
:solve kodkod x:0..2 & y:0..2 & x+y=2

[2018-05-28 12:36:06,209, T+7177542] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   x : 0 .. 2 & y : 0 .. 2 & x + y = 2  ints: irange(0,4), intatoms: none
[2018-05-28 12:36:06,212, T+7177545] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1,1,1]


TRUE

Solution:
	x = 2
	y = 0

We can find all solutions and check that we find exactly the three expected solutions:

In [24]:
:solve kodkod {x,y|x:0..2 & y:0..2 & x+y=2}=res

[2018-05-28 12:36:47,975, T+7219308] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   {x,y|x : 0 .. 2 & y : 0 .. 2 & x + y = 2} = res  ints: irange(0,4), intatoms: irange(0,2)
[2018-05-28 12:36:47,977, T+7219310] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1]


TRUE

Solution:
	res = {(0↦2),(1↦1),(2↦0)}

#### Translation to SMTLib ####
At iFM'2016 we (Krings et al.) presented a translation to SMTLib format to use either the Z3 or CVC4 SMT solver.
Compared to the Kodkod backend, no translation to SAT is performed, SMTLib supports integer predicates and operators in the language.
Here is ProB's SMTLib/Z3 API calls for the constraint:
- mk_var(integer,x) $\rightarrow$ 2
- mk_var(integer,y) $\rightarrow$ 3
- mk_int_const(0) $\rightarrow$ 4
- mk_op(greater_equal,2,4) $\rightarrow$ 5
- mk_int_const(2) $\rightarrow$ 6
- mk_op(greater_equal,6,2) $\rightarrow$ 7
- mk_int_const(0) $\rightarrow$ 8
- mk_op(greater_equal,3,8) $\rightarrow$ 9
- mk_int_const(2) $\rightarrow$ 10
- mk_op(greater_equal,10,3) $\rightarrow$ 11
- mk_op(add,2,3) $\rightarrow$ 12
- mk_int_const(2) $\rightarrow$ 13
- mk_op(equal,12,13) $\rightarrow$ 14
- mk_op_arglist(conjunct,[5,7,9,11,14]) $\rightarrow$ 15

In [25]:
:solve z3 x:0..2 & y:0..2 & x+y=2

TRUE

Solution:
	x = 0
	y = 2

#### ProB's CLP(FD) Solver ####
ProB's default solver makes use of constraint logic programming.
For arithmetic, it builts on top of CLP(FD), the finite domain library of SICStus Prolog.
In CLP(FD):
- every integer variable is associated with a domain of possible values, typically an interval
- when adding a new constraints, the domains of the involved variables are updated, or more precisely narrowed down.
- at some point we need to chose variables for enumeration; typically ProB chooses the value with the smallest domain.

Let us use a slightly adapted constraint ```x:0..9 & y:0..9 & x+y=2``` to illustrate how constraint processing works:

- x:0..9 $\leadsto$ x:0..9, y:$-\infty$..$\infty$
- y:0..9 $\leadsto$ x:0..9, y:0..9
- x+y=2 $\leadsto$ x:0..2, y:0..2
- Enumerate (label) variable x
 - x=0 $\leadsto$ x:0..0, y:2..2
 - x=1 $\leadsto$ x:1..1, y:1..1
 - x=2 $\leadsto$ x:2..2, y:0..0

In [33]:
:solve prob {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

In [31]:
:solve z3 {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [32]:
:solve kodkod {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

[2018-05-28 13:16:28,635, T+9599968] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   K : 1 .. 9 & P : 1 .. 9 & I : 0 .. 9 & S : 0 .. 9 ...  ints: irange(0,99980001), intatoms: irange(0,9)
[2018-05-28 13:16:28,636, T+9599969] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [679]


TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

Result for KISS*KISS=PASSION puzzle:

Solver | Runtime
-------|-------
ProB Default | 0.01 sec
Kodkod Backend | 1 sec
Z3 Backend | ? > 100 sec

### Unbounded integers ###
The SAT translation via Kodkod/Alloy requires to determine the bid width.
It cannot be applied to unbounded integers.
Even for bounded integers it is quite tricky to get the bid widths correct: one needs also to take care of intermediate results. Alloy can detect incorrect models where an overflow occured, but to our understanding not where an overflow prevented a model (e.g., use inside negation).

SMTLib is more tailored towards proof than towards model finding; as such it has typically no/less issues with unbounded values.
The ProB default solver can also deal with unbounded integers: it tries to narrow down domains to finite ones. If this fails, an unbounded variable is enumerated (partially) and an **enumeration warning** is generated. In case a solution is found, this warning is ignored, otherwise the result of ProB's analysis is **UNKNOWN**.
Some inconsistencies cannot be detected by interval/domain propagation; here it helps to activate ProB's CHR module which performs some additional inferences.

Let us perform some experiments.

In [34]:
:solve z3 x*x=100

TRUE

Solution:
	x = −10

Here is an example where ProB generates an enumeration warning, but finds a solution:

In [52]:
x>100 & x mod 2000 = 1 & x mod 3000 = 1

[2018-05-28 13:33:04,444, T+10595777] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : 6001:sup ---> 6001:6001


TRUE

Solution:
	x = 6001

In [53]:
:solve z3 x>100 & x mod 2000 = 1 & x mod 3000 = 1

TRUE

Solution:
	x = 6001

Here ProB generates an enumeration warning and does not find a solution, hence the result is **UNKNOWN**. Here Z3 finds a solution.

In [48]:
:solve prob x>100 & x mod 2000 = 1 & x mod 3000 = 1 & (x+x) mod 4501 = 0

[2018-05-28 13:31:19,432, T+10490765] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : 6001:sup ---> 6001:6001


CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [43]:
:solve z3 x>100 & x mod 2000 = 1 & x mod 3000 = 1 & (x+x) mod 4501 = 0

TRUE

Solution:
	x = 6756001

Here is an inconsistency which cannot be detected by CLP(FD)'s interval propagation. ProB can detect it with CHR enabled, but without the module the result is **UNKNOWN**.

In [45]:
:solve z3 x>y & y>x

FALSE

In [47]:
:solve prob x>y &y>x

[2018-05-28 13:31:04,544, T+10475877] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] % Timeout when posting constraint:
[2018-05-28 13:31:04,545, T+10475878] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] % kernel_objects:(_2097077#>0)
[2018-05-28 13:31:04,545, T+10475878] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : inf:sup ---> -1:3


CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

[2018-05-28 13:31:04,546, T+10475879] "ProB Output Logger for instance 712ca57b" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] % Timeout when posting constraint:


### Summary for Integer Arithmetic ###

Solver | Unbounded | Model Finding | Inconsistency Detection
------|------------|---------------|---
ProB CLP(FD) | yes | very good | limited with CHR
ProB Z3 | yes | reasonable | very good
ProB Kodkod | no | good | -
